this would actually end up working best at the bottom of the study so we can calulate + the causal model

theres some design stuff like per-protocol x per-zip code trials would be too small and g-method isnt entirely positive on that so its a bit weird? 

In [ ]:
#this should go at the end of study.ipynb 

In [ ]:
# =============================================================================
# FINAL CELL: CAUSAL IMPACT ANALYSIS & OPERATIONAL PRIORITIZATION
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery

# 1. Attach Causal Estimates to the Dataframe
# ---------------------------------------------------------
# We use the predictions from the AIPW/SuperLearner (preds dictionary)
# Risk Difference (RD) = Risk_with_Contrast (mu1) - Risk_without (mu0)
# If RD is NEGATIVE, contrast is PROTECTIVE.
# Gain from giving contrast = mu0 - mu1.

print("--- computing causal impact per patient ---")

analysis_df = df_final.copy()
analysis_df['mu1'] = preds['mu1']
analysis_df['mu0'] = preds['mu0']

# The "Benefit" of giving contrast (Prevented AKI)
# If mu1 < mu0, benefit is positive.
analysis_df['benefit_of_contrast'] = analysis_df['mu0'] - analysis_df['mu1']

# We only capture value by changing behavior for those who were WITHHELD (T=0)
# For those who already got contrast (T=1), the "gain" is 0 (we already did the right thing).
analysis_df['potential_gain'] = 0.0
analysis_df.loc[analysis_df['contrast_received'] == 0, 'potential_gain'] = \
    analysis_df.loc[analysis_df['contrast_received'] == 0, 'benefit_of_contrast']

# 2. Heuristic: "Excess Withholding" (The Baseline Logic)
# ---------------------------------------------------------
# Calculate global "sane" withholding rate (e.g., mean of all centers or top performing quartile)
global_withhold_rate = 1.0 - analysis_df['contrast_received'].mean()
analysis_df['expected_withheld'] = global_withhold_rate # Simple baseline

# 3. Fetch Department Names (if missing)
# ---------------------------------------------------------
# df_final usually drops strings. We need Department Names for the final plot.
if 'dept_name' not in analysis_df.columns:
    print("Fetching Department Names from BigQuery...")
    client = bigquery.Client()
    
    # Get unique visit IDs to minimize query
    vis_ids = analysis_df['visit_occurrence_id'].unique().tolist()
    
    # Chunked query to avoid limits
    def get_depts(ids, chunk_size=5000):
        rows = []
        for i in range(0, len(ids), chunk_size):
            chunk = ids[i:i+chunk_size]
            chunk_str = "(" + ",".join(map(str, chunk)) + ")"
            q = f"""
            SELECT v.visit_occurrence_id, c.concept_name as dept_name
            FROM `{os.environ["WORKSPACE_CDR"]}.visit_occurrence` v
            JOIN `{os.environ["WORKSPACE_CDR"]}.concept` c ON v.visit_concept_id = c.concept_id
            WHERE v.visit_occurrence_id IN {chunk_str}
            """
            rows.append(client.query(q).to_dataframe())
        return pd.concat(rows)
    
    dept_map = get_depts(vis_ids)
    analysis_df = analysis_df.merge(dept_map, on='visit_occurrence_id', how='left')

# Classify Depts
def clean_dept(name):
    n = str(name).lower()
    if 'emergency' in n: return 'ER'
    if 'inpatient' in n or 'hospital' in n or 'intensive' in n: return 'Inpatient'
    return 'Outpatient'

analysis_df['dept_group'] = analysis_df['dept_name'].apply(clean_dept)


# 4. Aggregation: Per-ZIP Impact
# ---------------------------------------------------------
# Group by ZIP
zip_agg = analysis_df.groupby('zip_code').agg({
    'contrast_received': 'count',                 # Total Volume
    'potential_gain': 'sum',                      # Total Causal AKI Prevented
    'mu0': 'mean'                                 # Average Baseline Risk
}).rename(columns={'contrast_received': 'total_volume', 'potential_gain': 'aki_prevented'})

# Calculate Withheld Counts
withheld_counts = analysis_df[analysis_df['contrast_received']==0].groupby('zip_code').size()
zip_agg['cnt_withheld'] = withheld_counts.reindex(zip_agg.index, fill_value=0)
zip_agg['withheld_rate'] = zip_agg['cnt_withheld'] / zip_agg['total_volume']

# Calculate Excess Withholding (Count vs Baseline)
# Excess = Observed Withheld - (Total * Global_Rate)
zip_agg['excess_withheld_count'] = (
    zip_agg['cnt_withheld'] - (zip_agg['total_volume'] * global_withhold_rate)
).clip(lower=0)

# Sort by Causal Impact (AKI Prevented)
zip_agg = zip_agg.sort_values('aki_prevented', ascending=False)
zip_agg['cum_impact'] = zip_agg['aki_prevented'].cumsum()
zip_agg['cum_pct'] = zip_agg['cum_impact'] / zip_agg['aki_prevented'].sum()

print("\nTop 10 High-Impact ZIPs (By Preventable AKI Events):")
print(zip_agg[['total_volume', 'cnt_withheld', 'withheld_rate', 'excess_withheld_count', 'aki_prevented', 'cum_pct']].head(10))


# 5. Aggregation: Per-Protocol Impact
# ---------------------------------------------------------
prot_agg = analysis_df.groupby('procedure_concept_id').agg({
    'potential_gain': 'sum',
    'contrast_received': 'count'
}).sort_values('potential_gain', ascending=False)
prot_agg['proc_name'] = prot_agg.index # Placeholder, map to names if desired
print("\nTop 5 High-Impact Protocols:")
print(prot_agg.head(5))


# 6. PLOTTING
# =========================================================
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(2, 2)

# Plot A: Pareto of Causal Impact
# -------------------------------
ax1 = fig.add_subplot(gs[0, 0])
top_n = 40
plot_data = zip_agg.head(top_n)

# Bar: Total Prevented Events
ax1.bar(plot_data.index.astype(str), plot_data['aki_prevented'], color='#d62728', alpha=0.8, label='Prevented AKI Cases')
ax1.set_ylabel('Projected AKI Cases Prevented', color='#d62728', fontsize=12)
ax1.tick_params(axis='x', rotation=90, labelsize=8)

# Line: Cumulative %
ax2 = ax1.twinx()
ax2.plot(plot_data.index.astype(str), plot_data['cum_pct'], color='black', marker='o', linewidth=2, label='Cumulative %')
ax2.axhline(0.5, color='blue', linestyle='--', label='50% Threshold')
ax2.set_ylabel('Cumulative % of Total Opportunity', fontsize=12)
ax2.set_ylim(0, 1.1)

ax1.set_title(f'Pareto of Preventable Harm (Top {top_n} ZIPs)\nMetric: Sum of Individual Causal Effects', fontsize=14)
ax1.legend(loc='upper left')
ax2.legend(loc='right')


# Plot B: Volume vs. Impact (The "Targeting" Map)
# -------------------------------
ax3 = fig.add_subplot(gs[0, 1])

# We plot Excess Withholding (Actionable Volume) vs AKI Prevented (Clinical Value)
sns.scatterplot(
    data=zip_agg,
    x='excess_withheld_count',
    y='aki_prevented',
    size='total_volume',
    sizes=(20, 500),
    alpha=0.6,
    hue='withheld_rate',
    palette='viridis',
    ax=ax3
)

# Add correlation line
m, b = np.polyfit(zip_agg['excess_withheld_count'], zip_agg['aki_prevented'], 1)
ax3.plot(zip_agg['excess_withheld_count'], m*zip_agg['excess_withheld_count'] + b, color='gray', linestyle='--', alpha=0.5)

ax3.set_title('Strategic Targeting: Actionable Volume vs. Clinical Yield', fontsize=14)
ax3.set_xlabel('Excess Withheld Procedures (Actionable Volume)')
ax3.set_ylabel('Total AKI Prevented (Causal Impact)')
ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Withhold Rate')


# Plot C: Department Breakdown of the "Bad 50%"
# -------------------------------
ax4 = fig.add_subplot(gs[1, :])

# Identify ZIPs that constitute the top 50% of the problem
target_zips = zip_agg[zip_agg['cum_pct'] <= 0.55].index.tolist()
print(f"\nAnalyzing {len(target_zips)} ZIPs accounting for ~50% of preventable AKI.")

# Filter data to these zips AND withheld cases
bad_actors = analysis_df[
    (analysis_df['zip_code'].isin(target_zips)) & 
    (analysis_df['contrast_received'] == 0)
].copy()

# Cross-tab
dept_counts = pd.crosstab(bad_actors['zip_code'], bad_actors['dept_group'])

# Sort by the order in the Pareto (Impact)
dept_counts = dept_counts.reindex(target_zips)

# Plot Stacked Bar
dept_counts.plot(kind='bar', stacked=True, colormap='coolwarm', ax=ax4, width=0.8)

ax4.set_title('Departmental Breakdown of Withheld Studies (Top 50% Impact ZIPs)', fontsize=14)
ax4.set_ylabel('Count of Withheld Procedures')
ax4.set_xlabel('ZIP Code (Sorted by Impact)')
ax4.legend(title='Department')

plt.tight_layout()
plt.show()

# Summary Print
print("--- SUMMARY STATISTICS ---")
print(f"Total Cohort Potential Gain: {zip_agg['aki_prevented'].sum():.1f} AKI events avoided.")
print(f"Impact Concentration: Top {len(target_zips)} ZIPs ({len(target_zips)/len(zip_agg):.1%} of sites) account for 50% of gains.")